In [3]:
import pandas as pd
from pandas.io.html import read_html
import re


def year(row): 
    """ iterates through the rows.If row has a year, returns year and updates the current year. If not, return the current year  """
    global current_year
    regexp = re.compile(r'\s(\d{4})$')
    if regexp.search(row):
        current_year = re.search(regexp, row).groups()[0]
        return current_year
    else :
        return current_year
    
def month(row):
    """ iterates through the rows.If row has a 3 char string i.e Jan, returns that string. If not, return the Jan """
    regexp = re.compile(r'(\w{3})')
    if regexp.search(row):
        current_month = re.search(regexp, row).groups()[0]
        return current_month
    else: 
        return "Jan"

def day(row):
    """ iterates through the rows.If row has a 2 digit, returns that string. If not, returns 1 """
    regexp = re.compile(r'(\d{2})')
    if regexp.search(row):
        current_day = re.search(regexp, row).groups()[0]
        return current_day
    else: 
        return "1"

def region(row):
    regexp = re.compile(r'.*PROVINCE.*')
    if regexp.search(row):
        return row
    
def city(row):
    regexp = re.compile(r'.*CITY|VILLAGE|SAIGON|HUẾ.*')
    if regexp.search(row):
        return row
    


# put wiki table into dataframe,rename columns, normalise Location data, change Year,Month,Day into Datetime column, reduce dataframe to date + Location dataframe

page = 'https://en.wikipedia.org/wiki/List_of_allied_military_operations_of_the_Vietnam_War_(1968)'
wikitables = pd.read_html(page, index_col=0, attrs={"class":"wikitable"}, skiprows=1)
df = wikitables[0]
df.columns = ["Operation","Description","Location","VC_KIA","Allied_KIA"]

df.index = df.index.rename("Date")
df = df.reset_index()
df["Location"] = df["Location"].str.upper()
current_year = "1968"
df["Year"] = df["Date"].apply(lambda x: year(x))
df["Month"] = df["Date"].apply(lambda x: month(x))
df["Day"] = df["Date"].apply(lambda x: day(x))
months = {"Jan" : "01" ,"Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug" :"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"}
df["Month"] = df["Month"].map(months)
df["Day"] = df["Day"].apply(lambda x: "01" if int(x) > 28 else x )
df['Date'] = df.apply(lambda x:datetime.strptime("{0} {1} {2}".format(x['Year'],x['Month'], x['Day']), "%Y %m %d")
                     ,axis=1)

df = df[["Date","Location","Description","VC_KIA","Allied_KIA"]]

df["Location"] = df["Location"].str.replace("SOUTH|EAST|NORTH|WEST|[0-9]|KM |VICINITY| OF |BETWEEN| IN |AROUND|CENTRAL ","")
df["Location"] = df["Location"].fillna("VIETNAM")
df["Region"] = df["Location"].apply(lambda x: region(x))
df["City"] = df["Location"].apply(lambda x: city(x))

print(df)

          Date                                           Location  \
0   1968-01-01  "PARROT'S BEAK" AREA SVAY RIENG PROVINCE, CAMB...   
1   1968-01-01                                            VIETNAM   
2   1968-01-01                                            VIETNAM   
3   1968-01-01                                            VIETNAM   
4   1973-01-19                                  HO CHI MINH TRAIL   
5   1973-01-19                                            I CORPS   
6   1973-01-25                                   KON TUM PROVINCE   
7   1973-01-01                               KHE SANH COMBAT BASE   
8   1973-01-10                                 PHƯỚC TUY PROVINCE   
9   1973-01-11                                  BIÊN HÒA PROVINCE   
10  1973-01-13                                  BIÊN HÒA PROVINCE   
11  1973-01-13                     GIA ĐỊNH AND LONG AN PROVINCES   
12  1973-01-14                                           IV CORPS   
13  1973-01-15                 QUA